<a href="https://colab.research.google.com/github/aishwarya-walimbe/Fraud-Detetection-Using-GNN/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
cur_path = "/content/drive/MyDrive/Fraud_Detection_Project/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Fraud_Detection_Project


In [4]:
!pip install torch_geometric
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, GATConv, BatchNorm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.9 MB/s eta 0:00:00


In [5]:
class ImprovedGraphSAGE(nn.Module):

    def __init__(self, in_channels: int, hidden_channels: int = 128,
                 dropout: float = 0.5):
        super().__init__()

        # --- Convolution layers ---
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)

        # --- Batch normalisation (one per layer) ---
        self.bn1 = BatchNorm(hidden_channels)
        self.bn2 = BatchNorm(hidden_channels)
        self.bn3 = BatchNorm(hidden_channels)

        # --- Skip-connection projection (input → hidden) ---
        # Needed because in_channels ≠ hidden_channels
        self.skip = nn.Linear(in_channels, hidden_channels, bias=False)

        # --- Final classifier head ---
        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2)
        )

        self.dropout = dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Layer 1
        h = self.conv1(x, edge_index)
        h = self.bn1(h)
        h = F.relu(h)
        h = F.dropout(h, p=self.dropout, training=self.training)

        # Skip: project raw input and add (residual)
        h = h + self.skip(x)

        # Layer 2
        h2 = self.conv2(h, edge_index)
        h2 = self.bn2(h2)
        h2 = F.relu(h2)
        h2 = F.dropout(h2, p=self.dropout, training=self.training)
        h2 = h2 + h                # residual from layer 1

        # Layer 3
        h3 = self.conv3(h2, edge_index)
        h3 = self.bn3(h3)
        h3 = F.relu(h3)
        h3 = F.dropout(h3, p=self.dropout, training=self.training)
        h3 = h3 + h2               # residual from layer 2

        return self.classifier(h3)

In [6]:
# Model 2 — Graph Attention Network  (uses edge features)
class GATFraudNet(nn.Module):

    def __init__(self, in_channels: int, hidden_channels: int = 64,
                 heads: int = 4, edge_dim: int = 5, dropout: float = 0.5):
        super().__init__()

        # heads=4 → 4 parallel attention mechanisms, results concatenated
        self.conv1 = GATConv(in_channels, hidden_channels,
                             heads=heads, edge_dim=edge_dim,
                             dropout=dropout, concat=True)

        # After concat: hidden_channels * heads input channels
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels,
                             heads=1, edge_dim=edge_dim,
                             dropout=dropout, concat=False)

        self.bn1 = BatchNorm(hidden_channels * heads)
        self.bn2 = BatchNorm(hidden_channels)

        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2)
        )

        self.dropout = dropout

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        h = self.conv1(x, edge_index, edge_attr=edge_attr)
        h = self.bn1(h)
        h = F.elu(h)                   # ELU works slightly better than ReLU for GAT
        h = F.dropout(h, p=self.dropout, training=self.training)

        h = self.conv2(h, edge_index, edge_attr=edge_attr)
        h = self.bn2(h)
        h = F.elu(h)

        return self.classifier(h)


# Quick test
if __name__ == "__main__":
    print("ImprovedGraphSAGE:", ImprovedGraphSAGE(in_channels=12, hidden_channels=128))
    print("\nGATFraudNet:      ", GATFraudNet(in_channels=12, hidden_channels=64,
                                              heads=4, edge_dim=5))


ImprovedGraphSAGE: ImprovedGraphSAGE(
  (conv1): SAGEConv(12, 128, aggr=mean)
  (conv2): SAGEConv(128, 128, aggr=mean)
  (conv3): SAGEConv(128, 128, aggr=mean)
  (bn1): BatchNorm(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (skip): Linear(in_features=12, out_features=128, bias=False)
  (classifier): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=64, out_features=2, bias=True)
  )
)

GATFraudNet:       GATFraudNet(
  (conv1): GATConv(12, 64, heads=4)
  (conv2): GATConv(256, 64, heads=1)
  (bn1): BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (classifier): Sequent